# Total energy Consumption Equation

The total energy consumption ( P ) can be represented as:

\[ P = \beta_1 \cdot N_{\text{servers}} + \beta_2 \cdot N_{\text{network\_switches\poe}} + \beta_3 \cdot N{\text{network\_switches\_non\poe}} + \beta_4 \cdot N{\text{hard\drives}} + \beta_5 \cdot N{\text{ssds}} \]

Where:

	•	 N_{\text{servers}} = Number of active servers
	•	 N_{\text{network\_switches\_poe}}  = Number of active network switches (PoE)
	•	 N_{\text{network\_switches\_non\_poe}}  = Number of active network switches (Non-PoE)
	•	 N_{\text{hard\_drives}}  = Number of active hard drives (HDD)
	•	 N_{\text{ssds}}  = Number of active solid-state drives (SSD)

And the coefficients are:

	•	 \beta_1 = 0.05279680365  kWh per server per 15 minutes
	•	 \beta_2  is a range between 25 Wh to 62.5 Wh per PoE switch per 15 minutes (converted to kWh)
	•	 \beta_3  is a range between 5 Wh to 12.5 Wh per non-PoE switch per 15 minutes (converted to kWh)
	•	 \beta_4  is a range between 0.175 Wh to 0.2625 Wh per HDD per 15 minutes (converted to kWh)
	•	 \beta_5 = 0.175  Wh per SSD per 15 minutes (converted to kWh)


In [7]:
from pyomo.environ import *

model = ConcreteModel()

# Decision Variables
model.N_servers = Var(domain=NonNegativeIntegers)
model.N_network_switches_poe = Var(domain=NonNegativeIntegers)
model.N_network_switches_non_poe = Var(domain=NonNegativeIntegers)
model.N_hard_drives = Var(domain=NonNegativeIntegers)
model.N_ssds = Var(domain=NonNegativeIntegers)

# Coefficients
beta1 = 0.05279680365
beta2_min, beta2_max = 25 / 1000 / 4, 62.5 / 1000 / 4  # convert Wh to kWh and per 15 min
beta3_min, beta3_max = 5 / 1000 / 4, 12.5 / 1000 / 4  # convert Wh to kWh and per 15 min
beta4_min, beta4_max = 0.175 / 1000 / 4, 0.2625 / 1000 / 4  # convert Wh to kWh and per 15 min
beta5 = 0.175 / 1000 / 4  # convert Wh to kWh and per 15 min

# Define ranges for coefficients to use the average for the constraints
beta2_avg = (beta2_min + beta2_max) / 2
beta3_avg = (beta3_min + beta3_max) / 2
beta4_avg = (beta4_min + beta4_max) / 2

# Total energy consumption constraint
model.total_energy_consumption = Constraint(expr=
    beta1 * model.N_servers +
    beta2_avg * model.N_network_switches_poe +
    beta3_avg * model.N_network_switches_non_poe +
    beta4_avg * model.N_hard_drives +
    beta5 * model.N_ssds
)

ERROR: Rule failed when generating expression for Constraint
total_energy_consumption with index None: ValueError: Constraint
'total_energy_consumption' does not have a proper value. Found
'0.05279680365*N_servers + 0.0109375*N_network_switches_poe +
0.0021875*N_network_switches_non_poe + 5.468750000000001e-05*N_hard_drives +
4.375e-05*N_ssds' Expecting a tuple or relational expression. Examples:
       sum(model.costs) == model.income (0, model.price[item], 50)
ERROR: Constructing component 'total_energy_consumption' from data=None
failed:
        ValueError: Constraint 'total_energy_consumption' does not have a
        proper value. Found '0.05279680365*N_servers +
        0.0109375*N_network_switches_poe +
        0.0021875*N_network_switches_non_poe +
        5.468750000000001e-05*N_hard_drives + 4.375e-05*N_ssds'
    Expecting a tuple or relational expression. Examples:
       sum(model.costs) == model.income (0, model.price[item], 50)


ValueError: Constraint 'total_energy_consumption' does not have a proper value. Found '0.05279680365*N_servers + 0.0109375*N_network_switches_poe + 0.0021875*N_network_switches_non_poe + 5.468750000000001e-05*N_hard_drives + 4.375e-05*N_ssds'
Expecting a tuple or relational expression. Examples:
   sum(model.costs) == model.income
   (0, model.price[item], 50)

To create a simple equation for the minimum number of active hardware based on the time of day, we can consider different times and the corresponding minimum requirements for each type of hardware. Let's define some time-based categories and their respective minimum requirements. For simplicity, let's divide the day into three periods: peak hours, off-peak hours, and regular hours.

### Time Periods:
1. **Peak Hours (e.g., 8 AM - 6 PM)**
2. **Regular Hours (e.g., 6 PM - 10 PM)**
3. **Off-Peak Hours (e.g., 10 PM - 8 AM)**

### Assumptions:
- Peak Hours require the highest number of active hardware.
- Regular Hours require a moderate number of active hardware.
- Off-Peak Hours require the lowest number of active hardware.

### Base Minimum Numbers:
These numbers should be adjusted based on your specific operational needs. For illustration:

- **Number of Servers:**
  - Peak Hours: 50
  - Regular Hours: 40
  - Off-Peak Hours: 30

- **Number of Network Switches (PoE):**
  - Peak Hours: 10
  - Regular Hours: 8
  - Off-Peak Hours: 6

- **Number of Network Switches (Non-PoE):**
  - Peak Hours: 5
  - Regular Hours: 4
  - Off-Peak Hours: 3

- **Number of Hard Drives (HDD):**
  - Peak Hours: 100
  - Regular Hours: 80
  - Off-Peak Hours: 60

- **Number of SSDs:**
  - Peak Hours: 50
  - Regular Hours: 40
  - Off-Peak Hours: 30

### Equations:
Let \( t \) be the time of day in hours (0-23).

1. **Number of Active Servers (\( N_s \))**:
\[ 
N_s = 
\begin{cases} 
50 & \text{if } 8 \leq t < 18 \\
40 & \text{if } 18 \leq t < 22 \\
30 & \text{if } 22 \leq t < 8 
\end{cases}
\]

2. **Number of Active Network Switches (PoE) (\( N_{ns\_poe} \))**:
\[ 
N_{ns\_poe} = 
\begin{cases} 
10 & \text{if } 8 \leq t < 18 \\
8 & \text{if } 18 \leq t < 22 \\
6 & \text{if } 22 \leq t < 8 
\end{cases}
\]

3. **Number of Active Network Switches (Non-PoE) (\( N_{ns\_non\_poe} \))**:
\[ 
N_{ns\_non\_poe} = 
\begin{cases} 
5 & \text{if } 8 \leq t < 18 \\
4 & \text{if } 18 \leq t < 22 \\
3 & \text{if } 22 \leq t < 8 
\end{cases}
\]

4. **Number of Active Hard Drives (HDD) (\( N_{hdd} \))**:
\[ 
N_{hdd} = 
\begin{cases} 
100 & \text{if } 8 \leq t < 18 \\
80 & \text{if } 18 \leq t < 22 \\
60 & \text{if } 22 \leq t < 8 
\end{cases}
\]

5. **Number of Active SSDs (\( N_{ssd} \))**:
\[ 
N_{ssd} = 
\begin{cases} 
50 & \text{if } 8 \leq t < 18 \\
40 & \text{if } 18 \leq t < 22 \\
30 & \text{if } 22 \leq t < 8 
\end{cases}
\]

Using these equations, you can adjust the minimum number of active hardware based on the time of day. You can tailor these numbers further based on your specific operational data and requirements.

In [ ]:
from pyomo.environ import ConcreteModel, Var, Constraint, NonNegativeIntegers, Objective, minimize

def get_min_active_hardware(time_of_day):
    if 8 <= time_of_day < 18:
        return {
            "servers": 50,
            "network_switches_poe": 10,
            "network_switches_non_poe": 5,
            "hard_drives": 100,
            "ssds": 50
        }
    elif 18 <= time_of_day < 22:
        return {
            "servers": 40,
            "network_switches_poe": 8,
            "network_switches_non_poe": 4,
            "hard_drives": 80,
            "ssds": 40
        }
    else:
        return {
            "servers": 30,
            "network_switches_poe": 6,
            "network_switches_non_poe": 3,
            "hard_drives": 60,
            "ssds": 30
        }

def create_model(time_of_day):
    min_hardware = get_min_active_hardware(time_of_day)
    
    model = ConcreteModel()
    
    # Define variables
    model.servers = Var(within=NonNegativeIntegers)
    model.network_switches_poe = Var(within=NonNegativeIntegers)
    model.network_switches_non_poe = Var(within=NonNegativeIntegers)
    model.hard_drives = Var(within=NonNegativeIntegers)
    model.ssds = Var(within=NonNegativeIntegers)
    
    # Define constraints based on minimum hardware requirements
    model.min_servers = Constraint(expr=model.servers >= min_hardware['servers'])
    model.min_network_switches_poe = Constraint(expr=model.network_switches_poe >= min_hardware['network_switches_poe'])
    model.min_network_switches_non_poe = Constraint(expr=model.network_switches_non_poe >= min_hardware['network_switches_non_poe'])
    model.min_hard_drives = Constraint(expr=model.hard_drives >= min_hardware['hard_drives'])
    model.min_ssds = Constraint(expr=model.ssds >= min_hardware['ssds'])
    
    # Example objective: Minimize total number of components
    model.objective = Objective(expr=model.servers + model.network_switches_poe + model.network_switches_non_poe + model.hard_drives + model.ssds, sense=minimize)
    
    return model

# Example usage:
time_of_day = 10  # Example time of day
model = create_model(time_of_day)

# Solve the model (you need a solver for this part)
from pyomo.opt import SolverFactory
solver = SolverFactory('glpk')  # Ensure you have the solver installed
result = solver.solve(model)

# Display results
model.servers.display()
model.network_switches_poe.display()
model.network_switches_non_poe.display()
model.hard_drives.display()
model.ssds.display()